In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


# **NLP DISASTER TWEET**

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
import os
import nltk

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import accuracy_score
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# **LOAD DATA**

In [37]:
df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [47]:
# Number of rows and columns
print('head',df.head())

# Data types of each column
data_types = df.dtypes

# Summary statistics
summary_stats = df.describe()

# Missing values
missing_values = df.isnull().sum()


print("rows,cols",num_rows,num_cols)
print('types=',data_types)
print('stats=',summary_stats)
print('missing values=',missing_values)

head    id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  
rows,cols 7613 5
types= id           int64
keyword     object
location    object
text        object
target       int64
dtype: object
stats=                  id      target
count   7613.000000  7613.00000
mean    5441.934848     0.42966
std     3137.116090     0.49506
min        1.000000     0.00000
25%     2734.000000     0.00000
50%     5408.000000     0.00000
75%     8146.000000     1.00000
max    10873.000000     1.00000
missing values= id             0

# **Data Preprocessing**

In [ ]:
# Remove URLs and special characters using a for loop
cleaned_text_list = []
for text in df['text']:
    text=text.lower()
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    # Remove "@" symbol
    text = text.replace('@', '')

    # Remove special characters (keep only alphanumeric characters and whitespace)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    # Uncomment either the stemming or lemmatization section based on your choice
    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    cleaned_text_list.append(' '.join(stemmed_tokens))
    
# Add the cleaned_text_list as a new column in the DataFrame
df['cleaned_text'] = cleaned_text_list
# data split
x=df['cleaned_text']
y=df['target']
x

# **Feature Extraction**

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
X= tfidf_vectorizer.fit_transform(df['cleaned_text'])
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

# **Classification**


In [49]:
cl_svc=SVC(kernel='linear')
cl_svc.fit(X_train, y_train)

cl_lr=LogisticRegression()
cl_lr.fit(X_train,y_train)

cl_knn = KNeighborsClassifier(n_neighbors=2)
cl_knn.fit(X_train, y_train)

cl_svc_y=cl_svc.predict(X_test)
cl_lr_y=cl_lr.predict(X_test)
cl_knn_y=cl_knn.predict(X_test)

svc_accuracy=accuracy_score(cl_svc_y,y_test)
lr_accuracy=accuracy_score(cl_lr_y,y_test)
knn_accuracy=accuracy_score(cl_knn_y,y_test)

print('svm:',classification_report (y_test,cl_svc_y))
print('logistic_Regression:',classification_report (y_test,cl_lr_y))
print('knn',classification_report (y_test,cl_knn_y))

train_svm_accuracy=cl_svc.score(X_train, y_train)
train_lr_accuracy=cl_lr.score(X_train, y_train)
train_knn_accuracy=cl_knn.score(X_train, y_train)

print("svc_Accuracy:",svc_accuracy )
#print("train_svm_Accuracy:", train_svm_accuracy)
print("logistic_Regression_Accuracy:",lr_accuracy )
#print("train_lr_Accuracy:", train_lr_accuracy)
print("knn_Accuracy:",knn_accuracy )
#print("train_knn_Accuracy:", train_knn_accuracy)

svm:               precision    recall  f1-score   support

           0       0.78      0.86      0.82       874
           1       0.78      0.68      0.73       649

    accuracy                           0.78      1523
   macro avg       0.78      0.77      0.77      1523
weighted avg       0.78      0.78      0.78      1523

logistic_Regression:               precision    recall  f1-score   support

           0       0.77      0.90      0.83       874
           1       0.82      0.65      0.72       649

    accuracy                           0.79      1523
   macro avg       0.80      0.77      0.78      1523
weighted avg       0.80      0.79      0.79      1523

knn               precision    recall  f1-score   support

           0       0.72      0.91      0.80       874
           1       0.81      0.53      0.64       649

    accuracy                           0.75      1523
   macro avg       0.77      0.72      0.72      1523
weighted avg       0.76      0.75      0.73 

# **Test Data Preprocessing**


In [40]:
# Remove URLs and special characters using a for loop
cleaned_test_list = []
for text in test['text']:
    text=text.lower()
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove "@" symbol
    text = text.replace('@', '')

    # Remove special characters (keep only alphanumeric characters and whitespace)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    cleaned_test_list.append(' '.join(stemmed_tokens))
    
# Add the cleaned_test_list as a new column in the DataFrame
test['cleaned_text'] = cleaned_test_list
test_x = tfidf_vectorizer.transform(test['cleaned_text'])

# **Predicting**

In [41]:
test_pred_svc=cl_svc.predict(test_x)
test_pred_lr=cl_lr.predict(test_x)
test_pred_knn=cl_knn.predict(test_x)

# **Result**

In [42]:
submissions = pd.read_csv(r"/kaggle/input/nlp-getting-started/sample_submission.csv")
submissions['target']=test_pred_svc
submissions.to_csv("submissions.csv", index=False)